In [41]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tti_explorer import utils

plt.rcParams.update({"figure.figsize": (12, 8)})

In [22]:
basedir = os.path.join(os.environ['DATA'], "tti-explorer", 'test-old-new')

## Test cases are the same

In [25]:
old_cases, old_meta = utils.load_cases(os.path.join(basedir, "new", "delve_seed0.json"))
new_cases, new_meta = utils.load_cases(os.path.join(basedir, "new-rng", "delve_seed0.json"))
# old_cases, old_meta = utils.load_cases(os.path.join(basedir, "old", "oxteam_seed0.json"))

In [26]:
def assert_contacts_equal(a, b):
    assert a.n_daily == b.n_daily
    for attr in ['home', 'work', 'other']:
        np.testing.assert_array_equal(getattr(a, attr), getattr(b, attr))

In [27]:
for (old_case, old_contacts), (new_case, new_contacts) in zip(old_cases, new_cases):
    assert old_case == new_case
    assert_contacts_equal(old_contacts, new_contacts)

AssertionError: 

In [30]:
def population_stats(case_contacts):
    case_outputs = list()
    for case, contacts in case_contacts:
        case_outputs.append(
            dict(
                under18=case.under18,
                covid=case.covid,
                symptomatic=case.symptomatic,
                day_noticed_symptoms=case.day_noticed_symptoms
            )
        )
    return pd.DataFrame(case_outputs), _

In [31]:
old_case_stats, _ = population_stats(old_cases)
new_case_stats, _ = population_stats(new_cases)

## Compare case statistics

Number of cases with certain properties

In [64]:
histogram = pd.concat(
    {
        "Old": old_case_stats.drop('day_noticed_symptoms', 1).sum(),
        "New": new_case_stats.drop('day_noticed_symptoms', 1).sum()
    },
    axis=1
)

histogram['Difference'] = histogram.diff(axis=1).dropna(axis=1)
histogram[f'Difference as % of total ({len(old_case_stats)/1000:.0f}k)'] = (
    histogram['Difference'] / len(old_case_stats)
).apply(
    lambda x: f"{100*x:.2f}%"
)

histogram    

,Old,New,Difference,Difference as % of total (100k)
under18,20852,20948,96.0,0.10%
covid,16473,16447,-26.0,-0.03%
symptomatic,93550,93366,-184.0,-0.18%


In [63]:
histogram = pd.concat({
    "Old": old_case_stats['day_noticed_symptoms'].value_counts(),
    "New": new_case_stats['day_noticed_symptoms'].value_counts()
    },
    axis=1
)

histogram['Difference'] = histogram.diff(axis=1).dropna(axis=1)
histogram[f'Difference as % of total ({len(old_case_stats)/1000:.0f}k)'] = (
    histogram['Difference'] / len(old_case_stats)
).apply(
    lambda x: f"{100*x:.2f}%"
)
histogram

,Old,New,Difference,Difference as % of total (100k)
-1,6450,6634,184.0,0.18%
1,23675,23203,-472.0,-0.47%
2,23256,23281,25.0,0.03%
3,18812,18821,9.0,0.01%
4,9349,9320,-29.0,-0.03%
5,4660,4789,129.0,0.13%
6,4709,4639,-70.0,-0.07%
7,4542,4678,136.0,0.14%
8,4547,4635,88.0,0.09%


## Test results are the same

In [6]:
new_res_dir = os.path.join(basedir, "new", "results")
old_res_dir = os.path.join(basedir, "old", "results")

In [7]:
assert os.listdir(new_res_dir) == os.listdir(new_res_dir)

In [8]:
new_all_res = pd.read_csv(os.path.join(new_res_dir, "all_results.csv"), index_col=0).drop('case_file', axis=1)
old_all_res = pd.read_csv(os.path.join(old_res_dir, "all_results.csv"), index_col=0).drop('case_file', axis=1)

In [9]:
new_means = new_all_res.query("statistic=='mean'").drop('statistic', axis=1)
new_stds = new_all_res.query("statistic=='std'").drop('statistic', axis=1)
old_means = old_all_res.query("statistic=='mean'").drop('statistic', axis=1)
old_stds = old_all_res.query("statistic=='std'").drop('statistic', axis=1)

In [10]:
delta = (new_means - old_means)
delta_stds = (new_stds - old_stds)

In [20]:
pd.testing.assert_frame_equal(new_all_res, old_all_res)

In [12]:
delta

,Base R,Effective R,# Manual Traces,# App Traces,# Tests Needed,# PersonDays Quarantined,% of Ongoing Transmission Prevented by Social Distancing,% of Ongoing Transmission Prevented by Isolating Cases with Symptoms and Quarantining Households,% of Ongoing Transmission Prevented by Tracing,% of Ongoing Transmission Allowed Through by Asymptomatic Cases Not Being Caught,% of Ongoing Transmission Allowed Through by Symptomatic Cases Not Complying,% of Ongoing Transmission Allowed Through by TTI Policy,% of Ongoing Transmission Prevented by Social Distancing - Symptomatic Compliant Only
scenario,,,,,,,,,,,,,
S5_no_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S5_symptom_based_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S5_test_based_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S5_test_based_TTI_test_contacts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4_no_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4_symptom_based_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4_test_based_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S4_test_based_TTI_test_contacts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
S3_no_TTI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from tti_explorer.utils import bool_bernoulli

In [14]:
import numpy as np
rs = np.random.RandomState(0)

In [15]:
p = 0.65    

In [16]:
rs = np.random.RandomState(0)
sample = rs.uniform(size=100000) < p

In [17]:
rs = np.random.RandomState(0)
sample2 = np.array([bool_bernoulli(p, rs) for _ in range(100000)])

In [18]:
sample.mean()

0.65051

In [19]:
sample2.mean()

0.65051